用于测试TPM文件中的gene_symbol列能否和pVACseq结果文件中的gene列匹配

In [1]:
import pandas as pd
import os


In [2]:
TPM_path='/work/longyh/BY/processed/GSE91061_BMS038109Sample.hg19KnownGene.tpm_with_symbol.csv'
tpm=pd.read_csv(TPM_path,index_col=0)

In [3]:
Pt3_path='/work/longyh/BY/processed/naive/Pt3_tumor.MHC_I.all_epitopes.aggregated.tsv'
pt3=pd.read_csv(Pt3_path,sep='\t')

In [4]:
pt3.columns

Index(['ID', 'Index', 'A*01:01', 'A*23:01', 'B*39:01', 'B*44:03', 'C*04:01',
       'C*12:03', 'Gene', 'AA Change', 'Num Passing Transcripts',
       'Best Peptide', 'Best Transcript', 'MANE Select', 'Canonical', 'TSL',
       'Allele', 'Pos', 'Prob Pos', 'Num Included Peptides',
       'Num Passing Peptides', 'IC50 MT', 'IC50 WT', '%ile MT', '%ile WT',
       'RNA Expr', 'RNA VAF', 'Allele Expr', 'RNA Depth', 'DNA VAF', 'Tier',
       'Evaluation'],
      dtype='object')

In [6]:
print(pt3.shape)
print(tpm.shape)

(184, 32)
(22187, 110)


In [10]:
#pt3的Gene列有多少能在tpm的gene_symbol中找到
matched_genes=pt3['Gene'].isin(tpm['gene_symbol'])
print(f"Number of matched genes: {matched_genes.sum()}")

Number of matched genes: 176
